In [ ]:
#!pip install torch torchvision transformers dinov2 tqdm pillow

from google.colab import drive
drive.mount('/content/drive')https://buysight-api.holocrow.com/mgmt/shelf_sight/transaction/251385/change/?_changelist_filters=p%3D1%26tenant__pk__exact%3D3

Mounted at /content/drive


In [ ]:
# Drive'daki klasör yolunu kendi yapına göre güncelle!
base_path = '/content/drive/MyDrive/ColabNotebooks/Dinov2'  # <<< burayı değiştir

train_dir = f'{base_path}/split_dataset/train'
val_dir   = f'{base_path}/split_dataset/val'
test_dir  = f'{base_path}/split_dataset/test'

batch_size = 32
num_epochs = 20

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_ds = datasets.ImageFolder(train_dir, transform=transform)
val_ds   = datasets.ImageFolder(val_dir, transform=transform)
test_ds  = datasets.ImageFolder(test_dir, transform=transform)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False)
test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False)

class_names = train_ds.classes
num_classes = len(class_names)
print("Sınıf isimleri:", class_names)

Sınıf isimleri: ['000', '001', '002', '003', '004']


In [ ]:
import torch
from transformers import Dinov2ForImageClassification

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

model = Dinov2ForImageClassification.from_pretrained(
    "facebook/dinov2-base",
    num_labels=num_classes,
    ignore_mismatched_sizes=True
).to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

Device: cpu


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of Dinov2ForImageClassification were not initialized from the model checkpoint at facebook/dinov2-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from tqdm import tqdm

def train(model, loader, optimizer, loss_fn, device):
    model.train()
    total_loss = 0
    for images, labels in tqdm(loader, desc="Train", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images).logits
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(loader)
    return avg_loss

def evaluate(model, loader, loss_fn, device):
    model.eval()
    correct, total, total_loss = 0, 0, 0
    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Eval", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).logits
            loss = loss_fn(outputs, labels)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            total_loss += loss.item()
    accuracy = correct / total
    avg_loss = total_loss / len(loader)
    return avg_loss, accuracy

In [ ]:
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}:")
    train_loss = train(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)
    print(f"  Train Loss: {train_loss:.4f}")
    print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

Epoch 1/20:


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-6-439943791.py", line 3, in <cell line: 0>
    train_loss = train(model, train_loader, optimizer, criterion, device)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-5-2646147258.py", line 6, in train
    for images, labels in tqdm(loader, desc="Train", leave=False):
  File "/usr/local/lib/python3.11/dist-packages/tqdm/std.py", line 1181, in __iter__
    for obj in iterable:
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 708, in __next__
    data = self._next_data()
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 764, in _next_data
    data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
         

TypeError: object of type 'NoneType' has no len()

In [ ]:
test_loss, test_acc = evaluate(model, test_loader, criterion, device)
print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")

save_path = f"{base_path}/dinov2_finetuned_car5.pth"
torch.save(model.state_dict(), save_path)
print("Model kaydedildi:", save_path)

KeyboardInterrupt: 

In [ ]:
import torch
from torchvision import datasets, transforms
from transformers import Dinov2ForImageClassification
from tqdm import tqdm
import os

# ---- YOL AYARLARI ----
base_path = '/content/drive/MyDrive/ColabNotebooks/Dinov2'  # Eğitimi nerede yaptıysan aynı olmalı
test_dir = f'{base_path}/split_dataset/test'

# ---- TRANSFORM ----
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ---- DATA LOADER ----
test_ds = datasets.ImageFolder(test_dir, transform=transform)
test_loader = torch.utils.data.DataLoader(test_ds, batch_size=32, shuffle=False)
class_names = test_ds.classes
num_classes = len(class_names)

print("Test sınıfları:", class_names)

# ---- MODEL YÜKLE ----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Dinov2ForImageClassification.from_pretrained(
    "facebook/dinov2-base",
    num_labels=num_classes,
    ignore_mismatched_sizes=True
).to(device)

model_path = f"{base_path}/dinov2_finetuned_car5.pth"
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

# ---- TEST ----
correct = 0
total = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Test seti üzerinde..."):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images).logits
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        all_preds += preds.cpu().tolist()
        all_labels += labels.cpu().tolist()

accuracy = correct / total
print(f"\nTest seti doğruluğu (accuracy): {accuracy:.4f} ({correct}/{total})")

Test sınıfları: ['000', '001', '002', '003', '004']


Some weights of Dinov2ForImageClassification were not initialized from the model checkpoint at facebook/dinov2-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Test seti üzerinde...:   0%|          | 0/10 [00:46<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
from PIL import Image

# Test setinden rastgele bir görsel için tahmin:
def predict(img_path, class_names):
    img = Image.open(img_path).convert("RGB")
    x = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(x).logits
        _, pred = torch.max(outputs, 1)
        return class_names[pred.item()]

# Örnek kullanım: test setinden bir görsel seç
example_image = os.path.join(test_dir, class_names[0], os.listdir(os.path.join(test_dir, class_names[0]))[0])
print("Gerçek sınıf:", class_names[0])
predict_image(example_image)

Gerçek sınıf: 000
Tahmin edilen sınıf: 000 (0)


'000'

In [ ]:
import torch
from torchvision import transforms
from transformers import Dinov2ForImageClassification
from PIL import Image
import os
import json
import numpy as np

# ----------- YOL AYARI (Kendi setup'ına göre değiştir!) ----------
base_path = "/content/drive/MyDrive/ColabNotebooks/Dinov2"
test_images_dir = f"{base_path}/test_images"   # test için klasör
model_ckpt_path = f"{base_path}/dinov2_finetuned_car5.pth" # model dosyan
labels_json_path = f"{base_path}/labels.json"

# ----------- SINIF İSİMLERİNİ YÜKLE -----------
with open(labels_json_path, "r") as f:
    label_map = json.load(f)
# İsim sıralama takibi için!
idx2label = [label_map[k] for k in sorted(label_map.keys())]
# Dilersen aşağıdaki gibi açıkça sıralayabilirsin (sıralama doğruysa değişme!):
# idx2label = ["Honda Civic", "Mitsubishi L200", "Opel Astra", "BMW 3 Serisi", "Toyota C-HR"]

# ----------- MODELİ YÜKLE -----------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Dinov2ForImageClassification.from_pretrained(
    "facebook/dinov2-base",
    num_labels=len(idx2label),
    ignore_mismatched_sizes=True
).to(device)
model.load_state_dict(torch.load(model_ckpt_path, map_location=device))
model.eval()

print(f"\n✅ Model ve checkpoint başarıyla yüklendi: {model_ckpt_path}\n")

# ----------- TRANSFORM -----------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ----------- TEST GÖRSELLERİNİ BUL -----------
image_paths = [os.path.join(test_images_dir, fname) for fname in os.listdir(test_images_dir) if fname.lower().endswith(('.jpg', '.jpeg', '.png'))]
image_paths.sort()
print(f"Found {len(image_paths)} images to test in '{os.path.basename(test_images_dir)}' folder.")
print("-" * 40)

# ----------- Eşik değeri belirle -----------
min_conf = 0.90   # %90 altında "bilinmeyen" verilecek

# ----------- TAHMİN FONKSİYONU ve ÇIKTI -----------
def pretty_scores(probabilities, idx2label):
    # Skorları en büyükten küçüğe sırala ve yazdır
    sorted_idx = np.argsort(probabilities)[::-1]
    rows = []
    for idx in sorted_idx:
        rows.append(f"    🔹 {idx2label[idx]:18} : %{probabilities[idx]*100:.2f}")
    return "\n" + "\n".join(rows)

for img_path in image_paths:
    img_name = os.path.basename(img_path)
    img = Image.open(img_path).convert("RGB")
    x = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        logits = model(x).logits.squeeze()
        probs = torch.softmax(logits, dim=0).cpu().numpy()
        top_idx = np.argmax(probs)
        pred_score = probs[top_idx]
        pred_label = idx2label[top_idx]

    print(f"\n📸 Test Sonuçları: {img_name}")

    if pred_score >= min_conf:
        print(f"  ✅ Tahmin: {pred_label} (Skor: %{pred_score*100:.2f})")
    else:
        print(f"  ⚠️  Tahmin: Bilinmeyen Araç (En Yakın Tahmin: {pred_label}, Skor: %{pred_score*100:.2f})")
        print(f"     (Sebep: En yüksek skor, %90 olan eşik değerinin altındadır)")

    print("  -- Tüm Skorlar --")
    print(pretty_scores(probs, idx2label))

print("-" * 40)
print("✅ Test işlemi tamamlandı.")

Some weights of Dinov2ForImageClassification were not initialized from the model checkpoint at facebook/dinov2-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



✅ Model ve checkpoint başarıyla yüklendi: /content/drive/MyDrive/ColabNotebooks/Dinov2/dinov2_finetuned_car5.pth

Found 25 images to test in 'test_images' folder.
----------------------------------------

📸 Test Sonuçları: bmw1.jpg
  ⚠️  Tahmin: Bilinmeyen Araç (En Yakın Tahmin: bmw-3-series, Skor: %75.25)
     (Sebep: En yüksek skor, %90 olan eşik değerinin altındadır)
  -- Tüm Skorlar --

    🔹 bmw-3-series       : %75.25
    🔹 opel-astra         : %18.57
    🔹 toyota-c-hr        : %6.05
    🔹 honda-civic        : %0.11
    🔹 mitsubishi-l200    : %0.03

📸 Test Sonuçları: bmw2.jpg
  ✅ Tahmin: honda-civic (Skor: %100.00)
  -- Tüm Skorlar --

    🔹 honda-civic        : %100.00
    🔹 mitsubishi-l200    : %0.00
    🔹 opel-astra         : %0.00
    🔹 bmw-3-series       : %0.00
    🔹 toyota-c-hr        : %0.00

📸 Test Sonuçları: bmw3.jpg
  ⚠️  Tahmin: Bilinmeyen Araç (En Yakın Tahmin: bmw-3-series, Skor: %54.11)
     (Sebep: En yüksek skor, %90 olan eşik değerinin altındadır)
  -- Tüm Skor

In [ ]:
print("Eğitim seti class_names:", train_ds.classes)
print("Test seti class_names:", test_ds.classes)
print("labels.json:", label_map)

Eğitim seti class_names: ['000', '001', '002', '003', '004']
Test seti class_names: ['000', '001', '002', '003', '004']
labels.json: {'000': 'honda-civic', '001': 'mitsubishi-l200', '002': 'opel-astra', '003': 'bmw-3-series', '004': 'toyota-c-hr'}


In [ ]:
for idx, name in enumerate(class_names):
    print(f"{idx}: {name} -> {label_map[name] if name in label_map else '???'}")

0: 000 -> honda-civic
1: 001 -> mitsubishi-l200
2: 002 -> opel-astra
3: 003 -> bmw-3-series
4: 004 -> toyota-c-hr
